
#Aspect Based Sentiment Analysis (ABSA)
Muneeza Iftikhar      - 012

Hafsa Hafeez Siddiqui - 026

Aqsa Khan             - 039


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/mindplatter.csv', header=None, names=['chapter'])

# Split the 'chapter' column into 'title' and 'text' columns
df[['title', 'text']] = df['chapter'].str.split(':', expand=True)

# Drop the original 'chapter' column
df = df.drop('chapter', axis=1)

print(df)

                      title                                               text
0                    Power    Respect the freedom that you were born with b...
1          Rest Your Heart    Your heart is precious, so take care of it. I...
2    Let the Mask Fall Off    I appreciate a genuine effort over a fake att...
3    Stay True to Yourself    Don't worry about what people think of you or...
4                Be Sought    Whatever you do, do it with purpose. Being fo...
5        Lifelong Learning    If you think that education only takes place ...
6            Take the Lead    You choose how your life is going to be. Don'...
7        Be Honest with Me    Don't tell me what I want to hear. Tell me th...
8   The Way They Treat You    Have you ever been told to treat people the w...
9     The Power of Silence    Silence can hold more meaning than words. It ...
10     Take Responsibility    Just as you can't deny that you can feel love...
11      Through Their Eyes    See yourself through t

In [ ]:
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

df.head()

,title,text
0,Power,Respect the freedom that you were born with but that you've denied yourself. Break free from the power that power has over you and believe in your own beliefs.
1,Rest Your Heart,"Your heart is precious, so take care of it. It may be able to forgive unconditionally, but don't exhaust it by surrounding yourself with those who constantly do you wrong. It may have an infinite capacity for patience, but don't exhaust it by surrounding yourself with those who don't value your time."
2,Let the Mask Fall Off,"I appreciate a genuine effort over a fake attempt to gain my trust. The clouds overtake the sky for a little while, but the sun always strikes through."
3,Stay True to Yourself,"Don't worry about what people think of you or about the way they try to make you feel. If people want to see you as a good person, they will. If people want to see you as a bad person, absolutely nothing you do will stop them. Ironically, the more you try to show them your good intentions, the more reason you give them to knock you down. Keep your head up high and be confident in what you do. Be confident in your intentions and keep your eyes ahead instead of wasting your time on those who want to drag you back. Because you can't change people's views, you have to believe that true change for yourself comes from within you, not from anyone else."
4,Be Sought,"Whatever you do, do it with purpose. Being focused is not something to be ashamed of. It is something to be proud of. When you know what you are doing and have a clear vision of where you are going, you will not need to chase opportunities. Opportunities will seek you. Happiness will chase you. And, instead of being a choice, you will be the one choosing."


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import subprocess

subprocess.run(["python", "-m", "spacy", "download", "en_core_web_lg"])

CompletedProcess(args=['python', '-m', 'spacy', 'download', 'en_core_web_lg'], returncode=0)

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import string
import nltk
from nltk.corpus import stopwords

def preprocess_text(text):
    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Remove abbreviations
    tokens = [token for token in tokens if "'" not in token]

    # Convert to lower case
    tokens = [token.lower() for token in tokens]

    # Remove punctuation marks
    punctuation = string.punctuation
    tokens = [token for token in tokens if token not in punctuation]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Enclose each token in quotes
    tokens = [f'"{token}"' for token in tokens]

    return tokens

# Apply the pre-processing function to the 'text' column
df['text'] = df['text'].apply(preprocess_text)

In [ ]:
import nltk

def pos_tagging(tokens):
    # POS Tagging
    tagged_tokens = nltk.pos_tag(tokens)

    return tagged_tokens

# Apply the POS tagging function to the 'text' column
df['tagged_text'] = df['text'].apply(pos_tagging)

In [ ]:
import spacy

# Load the spaCy model for English
nlp = spacy.load("en_core_web_lg")

def dependency_parsing(tagged_tokens):
    # Convert the list of tuples to a string
    sentence = " ".join([token[0] for token in tagged_tokens])

    # Parse the sentence using spaCy
    parsed_sentence = nlp(sentence)

    # Extract the dependencies
    dependencies = [(token.text, token.dep_, token.head.text) for token in parsed_sentence]

    return dependencies

# Apply the dependency parsing function to the 'text' column
df['dependencies'] = df['tagged_text'].apply(dependency_parsing)

In [ ]:
import spacy
from spacy.matcher import Matcher
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Load spaCy model
nlp = spacy.load("en_core_web_lg")

# Define patterns to match
pattern_adj_noun = [
    {"POS": "ADJ"},
    {"POS": "NOUN"}
]
pattern_vrb_adv = [
    {"POS": "VERB"},
    {"POS": "ADV"}
]

# Initialize Matcher
matcher = Matcher(nlp.vocab)

# Add patterns to the Matcher
matcher.add("ADJ_NOUN_PATTERN", [pattern_adj_noun])
matcher.add("VRB_ADV_PATTERN", [pattern_vrb_adv])

# Extract the chapter texts from the 'text' column
chapters = df['text'].tolist()

In [ ]:
# Load and preprocess text data
def preprocess_text(text):
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if token.is_alpha and token.lemma_ not in stopwords.words('english')]
    return ' '.join(tokens)

# Preprocess the chapter texts
preprocessed_texts = [preprocess_text(chapter) for chapter in chapters]

In [ ]:
# Extract aspects using the Matcher
def extract_aspects(text):
    doc = nlp(text)
    matches = matcher(doc)
    aspects = []
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        if string_id == "ADJ_NOUN_PATTERN" or string_id == "VRB_ADV_PATTERN":
            aspects.append(doc[start:end].text)
    return aspects

In [ ]:
# Sentiment analysis using VADER
def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    scores = sid.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'positive'
    elif scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Aspect-Based Sentiment Analysis (ABSA)
def absa(text):
    aspects = extract_aspects(text)
    aspect_sentiments = {aspect: get_sentiment(aspect) for aspect in aspects}
    return aspect_sentiments

# Perform ABSA on each chapter
chapter_aspects = {}
for chapter, preprocessed_text in zip(df['title'], preprocessed_texts):
    aspects = extract_aspects(preprocessed_text)
    aspect_sentiments = {aspect: get_sentiment(aspect) for aspect in aspects}
    chapter_aspects[chapter] = aspect_sentiments

# Display aspects and sentiments, segregated by chapter
for chapter, aspects in chapter_aspects.items():
    print(f"Chapter: {chapter}")
    for aspect, sentiment in aspects.items():
        print(f"Aspect: {aspect} (Sentiment Term: {aspect}, Sentiment Polarity: {sentiment})")
    print()  # Empty line for separation

# Prepare data for model training
aspect_sentiment_pairs = [(aspect, sentiment) for chapter_aspects in chapter_aspects.values() for aspect, sentiment in chapter_aspects.items()]
aspects, sentiments = zip(*aspect_sentiment_pairs)

Chapter: Power 
Aspect: free power (Sentiment Term: free power, Sentiment Polarity: positive)

Chapter: Rest Your Heart 
Aspect: precious take (Sentiment Term: precious take, Sentiment Polarity: positive)
Aspect: forgive unconditionally (Sentiment Term: forgive unconditionally, Sentiment Polarity: positive)
Aspect: surround value (Sentiment Term: surround value, Sentiment Polarity: positive)

Chapter: Let the Mask Fall Off 
Aspect: genuine effort (Sentiment Term: genuine effort, Sentiment Polarity: neutral)
Aspect: fake attempt (Sentiment Term: fake attempt, Sentiment Polarity: negative)
Aspect: little sun (Sentiment Term: little sun, Sentiment Polarity: neutral)

Chapter: Stay True to Yourself 
Aspect: good person (Sentiment Term: good person, Sentiment Polarity: positive)
Aspect: bad person (Sentiment Term: bad person, Sentiment Polarity: negative)
Aspect: stop ironically (Sentiment Term: stop ironically, Sentiment Polarity: negative)
Aspect: good intention (Sentiment Term: good inte

FEATURE EXTRACTION

In [ ]:
# Feature extraction using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(aspects)
y = sentiments

SPLITTING DATASET

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

NAIVE BAYES MODEL

In [ ]:
# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

EVALUATION METRICS & ACCURACY

In [ ]:
# Predict and evaluate
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.6923076923076923
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         1
     neutral       1.00      0.40      0.57         5
    positive       0.64      1.00      0.78         7

    accuracy                           0.69        13
   macro avg       0.55      0.47      0.45        13
weighted avg       0.73      0.69      0.64        13



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# ABSA on Chapters as a whole

In [ ]:
import csv

with open('absa_output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Chapter Name", "All Aspects", "All Sentiment Polarity"])  # header row

    for chapter, aspects in chapter_aspects.items():
        all_aspects = ",".join(aspects.keys())  # Join all aspects with commas
        all_sentiments = ",".join(aspects.values())  # Join all sentiments with commas
        writer.writerow([chapter, all_aspects, all_sentiments])

df_2 = pd.DataFrame([
    (chapter, ",".join(aspects.keys()), ",".join(aspects.values()))
    for chapter, aspects in chapter_aspects.items()
], columns=["Chapter Name", "All Aspects", "All Sentiment Polarity"])

df_2.to_csv("absa_output.csv", index=False)

In [ ]:
df_2.head(15)

,Chapter Name,All Aspects,All Sentiment Polarity,Overall Sentiment Polarity
0,Power,free power,positive,positive
1,Rest Your Heart,precious take forgive unconditionally surround value,"positive,positive,positive",positive
2,Let the Mask Fall Off,genuine effort fake attempt little sun,"neutral,negative,neutral",neutral
3,Stay True to Yourself,good person bad person stop ironically good intention confident intention drag back true change,"positive,negative,negative,positive,positive,negative,positive",positive
4,Be Sought,clear vision,positive,positive
5,Lifelong Learning,think long breathe mind dynamic knowledge subject area,"neutral,neutral,positive,neutral",neutral
6,Take the Lead,popular people cognizant decision,"positive,neutral",positive
7,Be Honest with Me,hurt definitely right person front word many time,"negative,neutral,neutral,neutral",neutral
8,The Way They Treat You,treat bad action good encourage listen also answer back good compare bad enemy,"positive,negative,positive,neutral,neutral,positive,negative",positive
9,The Power of Silence,heartless person innocent victim powerful word close mouth inner power,"negative,positive,positive,neutral,neutral",positive


In [ ]:
df_2['Overall Sentiment Polarity'] = ''


for index, row in df_2.iterrows():
    sentiments = row['All Sentiment Polarity'].split(',')


    sentiment_counts = {}
    for sentiment in sentiments:
        if sentiment in sentiment_counts:
            sentiment_counts[sentiment] += 1
        else:
            sentiment_counts[sentiment] = 1


    overall_sentiment = max(sentiment_counts, key=sentiment_counts.get)
    df_2.at[index, 'Overall Sentiment Polarity'] = overall_sentiment

In [ ]:
df_2

,Chapter Name,All Aspects,All Sentiment Polarity,Overall Sentiment Polarity
0,Power,free power,positive,positive
1,Rest Your Heart,"precious take,forgive unconditionally,surround value","positive,positive,positive",positive
2,Let the Mask Fall Off,"genuine effort,fake attempt,little sun","neutral,negative,neutral",neutral
3,Stay True to Yourself,"good person,bad person,stop ironically,good intention,confident intention,drag back,true change","positive,negative,negative,positive,positive,negative,positive",positive
4,Be Sought,clear vision,positive,positive
5,Lifelong Learning,"think long,breathe mind,dynamic knowledge,subject area","neutral,neutral,positive,neutral",neutral
6,Take the Lead,"popular people,cognizant decision","positive,neutral",positive
7,Be Honest with Me,"hurt definitely,right person,front word,many time","negative,neutral,neutral,neutral",neutral
8,The Way They Treat You,"treat well,bad action,good encourage,listen also,answer back,good compare,bad enemy","positive,negative,positive,neutral,neutral,positive,negative",positive
9,The Power of Silence,"heartless person,innocent victim,powerful word,close mouth,inner power","negative,positive,positive,neutral,neutral",positive


In [ ]:
pd.Series(" ".join(df_2['All Aspects']).split()).value_counts()[:40]

well         4
person       4
good         4
long         3
decision     3
            ..
strong       1
meanwhile    1
express      1
inside       1
keep         1
Name: count, Length: 40, dtype: int64

In [ ]:
#removing punctuations
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
html_tags = ['<br>']

for tag in html_tags:
	df_2['All Aspects'] = df_2['All Aspects'] .str.replace(tag, ' ')

for char in punc:
	df_2['All Aspects']  = df_2['All Aspects'] .str.replace(char, ' ')


other_stopwords = ['well','even']
df_2['All Aspects'] = df_2['All Aspects'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stopwords))

df_2.head(15)

,Chapter Name,All Aspects,All Sentiment Polarity,Overall Sentiment Polarity
0,Power,free power,positive,positive
1,Rest Your Heart,precious take forgive unconditionally surround value,"positive,positive,positive",positive
2,Let the Mask Fall Off,genuine effort fake attempt little sun,"neutral,negative,neutral",neutral
3,Stay True to Yourself,good person bad person stop ironically good intention confident intention drag back true change,"positive,negative,negative,positive,positive,negative,positive",positive
4,Be Sought,clear vision,positive,positive
5,Lifelong Learning,think long breathe mind dynamic knowledge subject area,"neutral,neutral,positive,neutral",neutral
6,Take the Lead,popular people cognizant decision,"positive,neutral",positive
7,Be Honest with Me,hurt definitely right person front word many time,"negative,neutral,neutral,neutral",neutral
8,The Way They Treat You,treat bad action good encourage listen also answer back good compare bad enemy,"positive,negative,positive,neutral,neutral,positive,negative",positive
9,The Power of Silence,heartless person innocent victim powerful word close mouth inner power,"negative,positive,positive,neutral,neutral",positive


In [ ]:
# Feature extraction using TF-IDF
aspects = df_2['All Aspects']
sentiments = df_2['Overall Sentiment Polarity']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(aspects)
y = sentiments

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [ ]:
# Train the model
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# Predict and evaluate
y_pred = model.predict(X_test)
print(f"Chapters as whole Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Chapters as whole Accuracy: 0.6666666666666666
              precision    recall  f1-score   support

     neutral       0.00      0.00      0.00         1
    positive       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
